In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
import os
import json
from datetime import datetime, timedelta
import pandas as pd
from tqdm import tqdm
import bspline
import bspline.splinelab as splinelab
import random
import sys
path = r'C:\Users\Yichen Jiang\Documents\PHD LIFE\Research\Hawkes Processes\Yelp'
sys.path.append(path)
from yelp_functions import *
from scipy.interpolate import splrep, BSpline

In [2]:
path = r'C:\Users\Yichen Jiang\Documents\PHD LIFE\Research\Hawkes Processes\Yelp'

path_business_withtext = os.path.join(path,'business with text features')

path_coefs = os.path.join(path,'dataframe_poisson_coefs')

path_preds = os.path.join(path,'dataframe_poisson_predictions')

filenames = os.listdir(path_business_withtext)

result_coefs = os.listdir(path_coefs)

result_preds = os.listdir(path_preds)

In [3]:
# list of deltas
list_deltas = [0.005, 0.05, 0.1, 1, 5]

# list of variables
list_variables = []
for i in range(0,5):
    list_variables.append(str(i+1)+'star')
list_variables += ['cool', 'funny', 'useful', 'average_stars', 'friend_count', 'elite_count', 'review_count', 'fan_count', 'yelping_since', 'mean_prob', 'mean_weight', 'sentiment_polarity', 'sentiment_subjectivity', 'text_length', 'textclean_length']

bspline_order = 3

knot_base = 'order' #'time' or 'order'

In [4]:
# create lookup dictionary and table
df_decaytable, dict_decaytable = create_decay_table(list_deltas)

# create dict_calendar for time-checking
dict_calendar = create_calendar()

In [5]:
def main(business_id,path,list_deltas,list_variables,bspline_order,knot_base):
    global dict_business
    global df_business
    global df_variables
    global df_bspline
    global dict_effective
    global dict_featurevalues
    global df_decaytable
    global dict_decaytable
    global dict_calendar
    global df_data
    global list_knot_order
    global list_knot_time
    
    with open(os.path.join(path,'business','food&restaurants',str(business_id)+'.json'),'r',encoding='utf-8') as file:
        for line in file.readlines():
            dict_business = json.loads(line)

    """ import user features"""
    get_user_features(business_id, path, dict_business)
    """ import text featrues"""
    get_text_features(business_id, path, dict_business)
    """ one-hot for star ratings"""
    one_hot_star(dict_business)

    """ str -> float or datetime"""
    for review_id in dict_business['Reviews']:
        for variable in dict_business['Reviews'][review_id].keys():
            if variable in list_variables:
                dict_business['Reviews'][review_id][variable] = float(dict_business['Reviews'][review_id][variable])
            elif variable == 'date':
                dict_business['Reviews'][review_id][variable] = datetime.strptime(dict_business['Reviews'][review_id][variable], "%Y-%m-%d %H:%M:%S")

    """ dictionary -> dataframe"""
    df_business = pd.DataFrame.from_dict(dict_business['Reviews']).T
    """ sort by datetime"""
    df_business = df_business.sort_values(by = 'date', axis = 0, ascending = True)

    """ add average_stars"""
    add_average_star(df_business)

    """ clear dataframe"""
    df_business = pd.concat([df_business['date'],df_business['stars'],df_business[list_variables]],axis=1)

    """ calculate decay values"""
    # dictionary for saving effective events (events with decay values > 1e-7)
    dict_effective = {}
    # dictionary for saving decay values for each event
    dict_decayvalues = {}
    
    calculate_decay_values(df_business, list_variables, list_deltas, dict_decaytable, dict_effective, dict_decayvalues)

    """ get dataframe """
    # get dataframe of independent variables and save it into .csv for running lasso model in R
    df_variables = pd.DataFrame(dict_decayvalues).T.drop('order',axis=1)

    """ get num of days for each event """

    list_date_simplified=[]
    for i in range(len(df_business.index)):
        date_num = get_date_num(df_business.iloc[i]['date'],df_business.iloc[0]['date'])
        list_date_simplified.append(date_num)

    df_business['date_num'] = list_date_simplified
    
    
    """ choose one review for all the reviews occurred in the same day, for poisson regression """
    """ count num of reviews for each day """
    date_num = 0
    count_reviews = 0
    dict_count = {}

    for i in tqdm(range(len(df_business.index))):
        curr_date_num = df_business.iloc[i]['date_num']
        # 1st review on current date
        if curr_date_num != date_num or i == 0:
            dict_count[curr_date_num] = {'num_review':0,
                      'num_1star':0,
                      'num_2star':0,
                      'num_3star':0,
                      'num_4star':0,
                      'num_5star':0,}
            index = df_business.iloc[[i]].index.values[0]
            # get star rating
            star = int(float(df_business.iloc[i]['stars']))
            # count
            dict_count[curr_date_num]['num_review'] += 1
            dict_count[curr_date_num]['num_'+str(star)+'star'] += 1
            if i == 0:
                df_variables_new = df_variables.loc[[index]]
            else:
                df_variables_new = pd.concat([df_variables_new,df_variables.loc[[index]]],axis=0)
            # update the date_num
            date_num = curr_date_num

        # more reviews on current date
        else:
            # get star rating
            star = int(float(df_business.iloc[i]['stars']))
            # count
            dict_count[curr_date_num]['num_review'] += 1
            dict_count[curr_date_num]['num_'+str(star)+'star'] += 1

    # add date_num column
    df_variables_new = pd.concat([df_variables_new,df_business['date_num']],axis=1,join='inner')

    """ transform the dataframe from review_id-base into date_num-base """
    # get data_range
    date_range = get_date_num(df_business.iloc[len(df_business)-1]['date'],df_business.iloc[0]['date'])+1

    for date_num in tqdm(range(date_range)):
        # if one or more review exist on current data_num
        if date_num in dict_count.keys():
            if date_num == 0:
                # get review
                df_temp = df_variables_new[df_variables_new.date_num == date_num]
                index = df_temp.index.values[0]
                # get all metrics
                df_temp = pd.concat([df_business.loc[[index]][['stars','date']],df_temp],axis=1,join='inner')
                df_temp.index = [date_num]
                df_data = df_temp
            else:
                df_temp = df_variables_new[df_variables_new.date_num == date_num]
                index = df_temp.index.values[0]
                # get all metrics
                df_temp = pd.concat([df_business.loc[[index]][['stars','date']],df_temp],axis=1,join='inner')
                df_data = pd.concat([df_data,df_temp],axis=0,ignore_index=True)
        else:
            # copy the previous one
            df_data = pd.concat([df_data,df_data.iloc[[len(df_data)-1]]],axis=0,ignore_index=True)
            df_data.loc[len(df_data)-1,'date_num'] = date_num
            dict_count[date_num] = {'num_review':0,
                      'num_1star':0,
                      'num_2star':0,
                      'num_3star':0,
                      'num_4star':0,
                      'num_5star':0,}
    
    """ convert dict_count into dataframe for dependent variables """
    df_count = pd.DataFrame(dict_count).T
    
    """ b spline basis function after removing unuseful events"""

    if knot_base == 'order':
        # get knot vector
        list_knot_order = get_knot_vector(df_data)
        # get b spline basis function
        df_data, df_bspline = b_spline_basis(bspline_order, list_knot_order, df_data)
    elif knot_base == 'time':
        # get knot vector
        list_knot_order, list_knot_time = get_knot_vector_time(df_data)
        # get b spline basis function
        df_data, df_bspline = b_spline_basis_time(bspline_order, list_knot_time, df_data)
    
    """ get dataframe """
    # get dataframe of independent variables and save it into .csv for running lasso model in R
    df_data = pd.concat([df_data,df_count,df_bspline],axis=1)
    df_data.to_csv(os.path.join(path,'dataframe_poisson' , 'df_'+str(business_id)+'_'+str(knot_base)+'.csv'), index=False, quoting=1)
    # export business_id
    file = open(os.path.join(path,'dataframe_poisson', 'business_id and knot_base.txt'),'r+')
    file.truncate()
    file.write(str(business_id)+','+str(knot_base)+'\n')
    file.close()   

In [12]:
for filename in filenames:
    try:
        print(filename.split('_with')[0])
        # update results
        result_preds = os.listdir(path_preds)
        if filename.endswith('.json'):
            if 'df_'+filename.split('_with')[0]+'_'+'order'+'_with predictions.csv' not in result_preds:
                knot_base = 'order'
                business_id = filename.split('_with')[0]
                print(business_id, knot_base)
                main(business_id,path,list_deltas,list_variables,bspline_order,knot_base)
            elif 'df_'+filename.split('_with')[0]+'_'+'order'+'_with predictions.csv' in result_preds:
                print(filename.split('_with')[0] + 'in order base has been finished')

            if 'df_'+filename.split('_with')[0]+'_'+'time'+'_with predictions.csv' not in result_preds:
                knot_base = 'time'
                business_id = filename.split('_with')[0]
                print(business_id, knot_base)
                main(business_id,path,list_deltas,list_variables,bspline_order,knot_base)
            elif 'df_'+filename.split('_with')[0]+'_'+'time'+'_with predictions.csv' in result_preds:
                print(filename.split('_with')[0] + 'in time base has been finished')    
    except KeyError:
        pass

--9e1ONYQuAa-CB_Rrw7Tw
--9e1ONYQuAa-CB_Rrw7Twin order base has been finished
--9e1ONYQuAa-CB_Rrw7Twin time base has been finished
-050d_XIor1NpCuWkbIVaQ
-050d_XIor1NpCuWkbIVaQin order base has been finished
-050d_XIor1NpCuWkbIVaQin time base has been finished
-3zffZUHoY8bQjGfPSoBKQ
-3zffZUHoY8bQjGfPSoBKQin order base has been finished
-3zffZUHoY8bQjGfPSoBKQin time base has been finished
-6h3K1hj0d4DRcZNUtHDuw
-6h3K1hj0d4DRcZNUtHDuwin order base has been finished
-6h3K1hj0d4DRcZNUtHDuwin time base has been finished
-6tvduBzjLI1ISfs3F_qTg
-6tvduBzjLI1ISfs3F_qTgin order base has been finished
-6tvduBzjLI1ISfs3F_qTgin time base has been finished
-7H-oXvCxJzuT42ky6Db0g
-7H-oXvCxJzuT42ky6Db0gin order base has been finished
-7H-oXvCxJzuT42ky6Db0gin time base has been finished
-95mbLJsa0CxXhpaNL4LvA
-95mbLJsa0CxXhpaNL4LvAin order base has been finished
-95mbLJsa0CxXhpaNL4LvAin time base has been finished
-a857YYdjzgOdOjFFRsRXQ
-a857YYdjzgOdOjFFRsRXQin order base has been finished
-a857YYdjzgOd

1ZnVfS-qP19upP_fwOhZsA
1ZnVfS-qP19upP_fwOhZsAin order base has been finished
1ZnVfS-qP19upP_fwOhZsAin time base has been finished
25wTaO0_Rk-GLb4k2uMs0Q
25wTaO0_Rk-GLb4k2uMs0Qin order base has been finished
25wTaO0_Rk-GLb4k2uMs0Qin time base has been finished
2DmTw0k3bcXuHdAyxO_bkw
2DmTw0k3bcXuHdAyxO_bkwin order base has been finished
2DmTw0k3bcXuHdAyxO_bkwin time base has been finished
2g4ZTD3ePNSbDAvh6qAOKQ
2g4ZTD3ePNSbDAvh6qAOKQin order base has been finished
2g4ZTD3ePNSbDAvh6qAOKQin time base has been finished
2GmGT-7QjowR1ihup3FbVA
2GmGT-7QjowR1ihup3FbVAin order base has been finished
2GmGT-7QjowR1ihup3FbVAin time base has been finished
2gUbgbdJ7IFSbicBXlSchw
2gUbgbdJ7IFSbicBXlSchwin order base has been finished
2gUbgbdJ7IFSbicBXlSchwin time base has been finished
2iTsRqUsPGRH1li1WVRvKQ
2iTsRqUsPGRH1li1WVRvKQin order base has been finished
2iTsRqUsPGRH1li1WVRvKQin time base has been finished
2KWkeOk1KaH50Es_A-8cnA
2KWkeOk1KaH50Es_A-8cnAin order base has been finished
2KWkeOk1KaH50

6t7mlTVjDA9fhuFAZmVzAQ
6t7mlTVjDA9fhuFAZmVzAQin order base has been finished
6t7mlTVjDA9fhuFAZmVzAQin time base has been finished
6tSvz_21BMo3a4GaItwa0g
6tSvz_21BMo3a4GaItwa0gin order base has been finished
6tSvz_21BMo3a4GaItwa0gin time base has been finished
6Vyp-9i1jzwK_bvisuPJpQ
6Vyp-9i1jzwK_bvisuPJpQin order base has been finished
6Vyp-9i1jzwK_bvisuPJpQin time base has been finished
6V_7ldr9tjmet0BdAnC34A
6V_7ldr9tjmet0BdAnC34Ain order base has been finished
6V_7ldr9tjmet0BdAnC34Ain time base has been finished
6xgcHeLad-VaoTIQewK84A
6xgcHeLad-VaoTIQewK84Ain order base has been finished
6xgcHeLad-VaoTIQewK84Ain time base has been finished
6ZIHxvFTHC1pvAzAS0uLDA
6ZIHxvFTHC1pvAzAS0uLDAin order base has been finished
6ZIHxvFTHC1pvAzAS0uLDAin time base has been finished
73_UT7fZ7mzXcguX8-oSuQ
73_UT7fZ7mzXcguX8-oSuQin order base has been finished
73_UT7fZ7mzXcguX8-oSuQin time base has been finished
77h11eWv6HKJAgojLx8G4w
77h11eWv6HKJAgojLx8G4win order base has been finished
77h11eWv6HKJA

aD0EslPREWGDlSpD0RMlEg
aD0EslPREWGDlSpD0RMlEgin order base has been finished
aD0EslPREWGDlSpD0RMlEgin time base has been finished
ADgacmZ-qXrSOhMfU6bmTA
ADgacmZ-qXrSOhMfU6bmTAin order base has been finished
ADgacmZ-qXrSOhMfU6bmTAin time base has been finished
AfN3Z1U6QPEgAb5F2CQm8w
AfN3Z1U6QPEgAb5F2CQm8win order base has been finished
AfN3Z1U6QPEgAb5F2CQm8win time base has been finished
AhPmh0NaRDbEYyhd9egzLA
AhPmh0NaRDbEYyhd9egzLAin order base has been finished
AhPmh0NaRDbEYyhd9egzLAin time base has been finished
ahSFUPojs9X3-1jP-QPb-w
ahSFUPojs9X3-1jP-QPb-win order base has been finished
ahSFUPojs9X3-1jP-QPb-win time base has been finished
aiX_WP7NKPTdF9CfI-M-wg
aiX_WP7NKPTdF9CfI-M-wgin order base has been finished
aiX_WP7NKPTdF9CfI-M-wgin time base has been finished
ajoqEHnCZTD8-8GqGLq9-Q
ajoqEHnCZTD8-8GqGLq9-Qin order base has been finished
ajoqEHnCZTD8-8GqGLq9-Qin time base has been finished
AjWFFZN8KwyqJzzi0DyEsw
AjWFFZN8KwyqJzzi0DyEswin order base has been finished
AjWFFZN8KwyqJ

C8D_GU9cDDjbOJfCaGXxDQin time base has been finished
C9ImzBi5fn742ZcAYDww2A
C9ImzBi5fn742ZcAYDww2Ain order base has been finished
C9ImzBi5fn742ZcAYDww2Ain time base has been finished
CauQnqZ5eowyrr7oWF_p3Q
CauQnqZ5eowyrr7oWF_p3Qin order base has been finished
CauQnqZ5eowyrr7oWF_p3Qin time base has been finished
CCRgGKhzxSpl38w7yhqKUw
CCRgGKhzxSpl38w7yhqKUwin order base has been finished
CCRgGKhzxSpl38w7yhqKUwin time base has been finished
cHdJXLlKNWixBXpDwEGb_A
cHdJXLlKNWixBXpDwEGb_Ain order base has been finished
cHdJXLlKNWixBXpDwEGb_Ain time base has been finished
CiYLq33nAyghFkUR15pP-Q
CiYLq33nAyghFkUR15pP-Qin order base has been finished
CiYLq33nAyghFkUR15pP-Qin time base has been finished
cJWbbvGmyhFiBpG_5hf5LA
cJWbbvGmyhFiBpG_5hf5LAin order base has been finished
cJWbbvGmyhFiBpG_5hf5LAin time base has been finished
cKdox2gt3L1Dbb7MpOPdWg
cKdox2gt3L1Dbb7MpOPdWgin order base has been finished
cKdox2gt3L1Dbb7MpOPdWgin time base has been finished
cKgUCzMGuRgkbKXUsgeXUw
cKgUCzMGuRgkbK

...

RhEvP5flF6KoPriMHmVYGgin time base has been finished
riFzCvp77DMKDX-5GoTpqA
riFzCvp77DMKDX-5GoTpqAin order base has been finished
riFzCvp77DMKDX-5GoTpqAin time base has been finished
riM-pBsdqRRVXtAVbLfsxQ
riM-pBsdqRRVXtAVbLfsxQin order base has been finished
riM-pBsdqRRVXtAVbLfsxQin time base has been finished
rioQ_p2pILNbJ4Xp5jW6-Q
rioQ_p2pILNbJ4Xp5jW6-Qin order base has been finished
rioQ_p2pILNbJ4Xp5jW6-Qin time base has been finished
RixnVQV24bc3p92mSntaVA
RixnVQV24bc3p92mSntaVAin order base has been finished
RixnVQV24bc3p92mSntaVAin time base has been finished
Rj-7ymdw8aNZBRqGRAjR3Q
Rj-7ymdw8aNZBRqGRAjR3Qin order base has been finished
Rj-7ymdw8aNZBRqGRAjR3Qin time base has been finished
RJNAeNA-209sctUO0dmwuA
RJNAeNA-209sctUO0dmwuAin order base has been finished
RJNAeNA-209sctUO0dmwuAin time base has been finished
RNi6tW22UMgHwWLAb0mYdA
RNi6tW22UMgHwWLAb0mYdAin order base has been finished
RNi6tW22UMgHwWLAb0mYdAin time base has been finished
roR1_DDlMk7mhi5_R6XVKA
roR1_DDlMk7mhi

SZEFE5hL7aN5nM-A44iPwQin time base has been finished
T-KniGykrZ46ZC9plOTspw
T-KniGykrZ46ZC9plOTspwin order base has been finished
T-KniGykrZ46ZC9plOTspwin time base has been finished
T0Uw6vwwfO3el29wBoDamQ
T0Uw6vwwfO3el29wBoDamQin order base has been finished
T0Uw6vwwfO3el29wBoDamQin time base has been finished
T2tEMLpTeSMxLKpxwFdS3g
T2tEMLpTeSMxLKpxwFdS3gin order base has been finished
T2tEMLpTeSMxLKpxwFdS3gin time base has been finished
t33qVzQ6AdSiq4lnI0KOyg
t33qVzQ6AdSiq4lnI0KOygin order base has been finished
t33qVzQ6AdSiq4lnI0KOygin time base has been finished
t53w4JlkxTau-G-TzhSWog
t53w4JlkxTau-G-TzhSWogin order base has been finished
t53w4JlkxTau-G-TzhSWogin time base has been finished
tcfL9_Qe-12eX4rdmK_6XA
tcfL9_Qe-12eX4rdmK_6XAin order base has been finished
tcfL9_Qe-12eX4rdmK_6XAin time base has been finished
TCoBE_BkDRrK0bWrh5VueQ
TCoBE_BkDRrK0bWrh5VueQin order base has been finished
TCoBE_BkDRrK0bWrh5VueQin time base has been finished
TCWMgOiV0PxQkWE1SyBIWQ
TCWMgOiV0PxQkW

utoTa3Ngv_oSfB5TIMgN_Qin time base has been finished
UTXNr62dGSK-tet8OeXUcQ
UTXNr62dGSK-tet8OeXUcQin order base has been finished
UTXNr62dGSK-tet8OeXUcQin time base has been finished
uuGL8diLlHfeUeFuod3F-w
uuGL8diLlHfeUeFuod3F-win order base has been finished
uuGL8diLlHfeUeFuod3F-win time base has been finished
UUGoM4q4i8rK2CBRS0xDAw
UUGoM4q4i8rK2CBRS0xDAwin order base has been finished
UUGoM4q4i8rK2CBRS0xDAwin time base has been finished
UV2Jt8slktGu14gLZeNCjA
UV2Jt8slktGu14gLZeNCjAin order base has been finished
UV2Jt8slktGu14gLZeNCjAin time base has been finished
uW6UHfONAmm8QttPkbMewQ
uW6UHfONAmm8QttPkbMewQin order base has been finished
uW6UHfONAmm8QttPkbMewQin time base has been finished
uWECX6-Uq9n8v5ipk9R29A
uWECX6-Uq9n8v5ipk9R29A order
Uxm9N6jwmJybOv1NcBbnRw
Uxm9N6jwmJybOv1NcBbnRwin order base has been finished
Uxm9N6jwmJybOv1NcBbnRwin time base has been finished
UYh1N1xQhOh-a7nX92xPzQ
UYh1N1xQhOh-a7nX92xPzQin order base has been finished
UYh1N1xQhOh-a7nX92xPzQin time base has

wTjvgkUJMVWrYed9mXdQjgin order base has been finished
wTjvgkUJMVWrYed9mXdQjgin time base has been finished
wUKzaS1MHg94RGM6z8u9mw
wUKzaS1MHg94RGM6z8u9mwin order base has been finished
wUKzaS1MHg94RGM6z8u9mwin time base has been finished
wuQDMDlqM17jQNo0lYQZ7g
wuQDMDlqM17jQNo0lYQZ7gin order base has been finished
wuQDMDlqM17jQNo0lYQZ7gin time base has been finished
wWuGa3OOQJro_XGe-GKBXA
wWuGa3OOQJro_XGe-GKBXAin order base has been finished
wWuGa3OOQJro_XGe-GKBXAin time base has been finished
wX1ORSDKPkA5ftE7F6UONw
wX1ORSDKPkA5ftE7F6UONwin order base has been finished
wX1ORSDKPkA5ftE7F6UONwin time base has been finished
wxL9wgxLeuMMfI6SAXIzJw
wxL9wgxLeuMMfI6SAXIzJwin order base has been finished
wxL9wgxLeuMMfI6SAXIzJwin time base has been finished
Wxxvi3LZbHNIDwJ-ZimtnA
Wxxvi3LZbHNIDwJ-ZimtnAin order base has been finished
Wxxvi3LZbHNIDwJ-ZimtnAin time base has been finished
wzugmCevnXuCMCF4upAf0w
wzugmCevnXuCMCF4upAf0win order base has been finished
wzugmCevnXuCMCF4upAf0win time base h

yMopev-RPxfEXgqkJOvCGAin order base has been finished
yMopev-RPxfEXgqkJOvCGAin time base has been finished
YmOWkved7K8q3eXgf4NNnA
YmOWkved7K8q3eXgf4NNnAin order base has been finished
YmOWkved7K8q3eXgf4NNnAin time base has been finished
YnmtUJGqQIQL9tz1MRyfqA
YnmtUJGqQIQL9tz1MRyfqAin order base has been finished
YnmtUJGqQIQL9tz1MRyfqAin time base has been finished
yNPh5SO-7wr8HPpVCDPbXQ
yNPh5SO-7wr8HPpVCDPbXQin order base has been finished
yNPh5SO-7wr8HPpVCDPbXQin time base has been finished
yNVcnSaMSrTmjJBMrHTy9w
yNVcnSaMSrTmjJBMrHTy9win order base has been finished
yNVcnSaMSrTmjJBMrHTy9win time base has been finished
yp2nRId4v-bDtrYl5A3F-g
yp2nRId4v-bDtrYl5A3F-gin order base has been finished
yp2nRId4v-bDtrYl5A3F-gin time base has been finished
YPavuOh2XsnRbLfl0DH2lQ
YPavuOh2XsnRbLfl0DH2lQin order base has been finished
YPavuOh2XsnRbLfl0DH2lQin time base has been finished
YQ--LJ7pvjiDSqNv0TuKTQ
YQ--LJ7pvjiDSqNv0TuKTQin order base has been finished
YQ--LJ7pvjiDSqNv0TuKTQin time base h

In [8]:
result_preds

['df_--9e1ONYQuAa-CB_Rrw7Tw_order_with predictions.csv',
 'df_--9e1ONYQuAa-CB_Rrw7Tw_time_with predictions.csv',
 'df_-050d_XIor1NpCuWkbIVaQ_order_with predictions.csv',
 'df_-050d_XIor1NpCuWkbIVaQ_time_with predictions.csv',
 'df_-3zffZUHoY8bQjGfPSoBKQ_order_with predictions.csv',
 'df_-3zffZUHoY8bQjGfPSoBKQ_time_with predictions.csv',
 'df_-6h3K1hj0d4DRcZNUtHDuw_order_with predictions.csv',
 'df_-6h3K1hj0d4DRcZNUtHDuw_time_with predictions.csv',
 'df_-6tvduBzjLI1ISfs3F_qTg_order_with predictions.csv',
 'df_-6tvduBzjLI1ISfs3F_qTg_time_with predictions.csv',
 'df_-7H-oXvCxJzuT42ky6Db0g_order_with predictions.csv',
 'df_-7H-oXvCxJzuT42ky6Db0g_time_with predictions.csv',
 'df_-95mbLJsa0CxXhpaNL4LvA_order_with predictions.csv',
 'df_-95mbLJsa0CxXhpaNL4LvA_time_with predictions.csv',
 'df_-a857YYdjzgOdOjFFRsRXQ_order_with predictions.csv',
 'df_-a857YYdjzgOdOjFFRsRXQ_time_with predictions.csv',
 'df_-av1lZI1JDY_RZN2eTMnWg_order_with predictions.csv',
 'df_-av1lZI1JDY_RZN2eTMnWg_time_with p